In [ ]:
#iteration 1: just preprocess tweets and generate topic models (all languages)

In [45]:
category = 2

In [46]:
import pandas as pd 
import numpy as np
import preprocessor as p
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

import pandas as pd

df = pd.read_csv(f"/mnt/smdata/twitter/VAMoS/finalData/ALLcat{category}.csv", engine='python')
print(df.shape)

#removing stopwords
all_languages = ['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek','hungarian', 'indonesian', 
 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']

stops = set(stopwords.words(all_languages))
print("total stopwords in all languages = "+ str(len(list(stops))))

vectorizer_model = CountVectorizer(analyzer = 'word', stop_words=list(stops))

df = df.drop_duplicates(subset=['text'])
print("after removing duplicate tweets:")
print(df.shape)
#convert to list 
tweets = df.text.to_list()
print("total tweets = "+str(len(tweets)))

# remove URLs, mentions, emojis, RT etc.
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY)

cleaned_tweets = []
for i in tweets:
    ct = p.clean(i)
    cleaned_tweets.append(ct)

"""
k=1
for tweet in cleaned_tweets:
    print(k)
    print(tweet)
    k=k+1
"""

(32178, 3)
total stopwords in all languages = 6715
after removing duplicate tweets:
(26224, 3)
total tweets = 26224


'\nk=1\nfor tweet in cleaned_tweets:\n    print(k)\n    print(tweet)\n    k=k+1\n'

In [47]:
topic_model = BERTopic(nr_topics="auto", vectorizer_model=vectorizer_model, language="multilingual")
topics, probabilities = topic_model.fit_transform(cleaned_tweets)

tot_topics = len(topic_model.get_topics()) - 1
print("total topics = " + str(tot_topics))
topic_model.get_topic_freq().head(11)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count
0,-1,12364
1,0,1505
2,1,446
3,2,387
4,3,339
5,4,332
6,5,319
7,6,286
8,7,235
9,8,219


In [48]:
topic_model.visualize_barchart(top_n_topics=12,n_words=10,title=f"Cat {category} (# unique tweets = {len(tweets)}) : Topic word scores (total = {tot_topics} )")

In [49]:
topic_model.save(f"/mnt/smdata/twitter/VAMoS/topicModels/final_cat{category}_model")
print("model saved")
df = topic_model.get_document_info(cleaned_tweets)
df.to_csv(f"topicModels/topicinfo_cat{category}.csv")
print("doc info saved")

/home/ubuntu/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



model saved
doc info saved


In [50]:
#reduce the number of topics after training the model. This is a great option if retraining the model will take many hours
#new_topics, new_probs = topic_model.reduce_topics(cleaned_tweets, nr_topics=250)

In [51]:
#!pip install -U kaleido